In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats


fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/box/"

In [21]:
cols1 = ['TEAM_NAME', 'GP', 'W', 'L', 'W_PCT',
       'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'EFG_PCT', 'TS_PCT', 'E_PACE', 'PACE', 'POSS',
       "W_RANK", "L_RANK",
       'W_PCT_RANK', 'OFF_RATING_RANK', 'DEF_RATING_RANK',
       'NET_RATING_RANK',]

In [28]:
dfa = []
for year in range(2001,2024):
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season = get_ss(year)
    )
    df1 = stats.get_data_frames()[0]
    df1 = df1.sort_values("W_PCT",ascending=False).reset_index(drop=True)
    time.sleep(0.6)
    stats = leaguedashteamstats.LeagueDashTeamStats(
        measure_type_detailed_defense="Advanced",
        season_type_all_star="Playoffs",
        season=get_ss(year)
    )
    df2 = stats.get_data_frames()[0]
    df2 = df2.sort_values("W",ascending=False).reset_index(drop=True)
    df1 = df1[cols1]
    df2 = df2[cols1]
    df3 = pd.merge(df2,df1,on=["TEAM_NAME"],how="left")
    df3["season"] = year + 1
    
    time.sleep(0.6)
    stats = leaguegamelog.LeagueGameLog(
        player_or_team_abbreviation="T",
        season=get_ss(year),
        season_type_all_star="Playoffs",
    )
    df4 = stats.get_data_frames()[0]
    df4["Round"] = df4["GAME_ID"].str[-3]
    df4["Win"] = df4["WL"] == "W"
    df4["Loss"] = df4["WL"] == "L"
    df5 = df4.groupby(["TEAM_NAME","Round"])[["Win","Loss"]].sum().reset_index()
    df6 = df5.groupby(["TEAM_NAME"])[["Round"]].count().reset_index()
    df7 = pd.merge(df3,df6,on="TEAM_NAME",how="left")
    dfa.append(df7)
    time.sleep(0.6)
df8 = pd.concat(dfa)
df8 = df8.rename(columns={"Round":"Round_x"})

In [29]:
df8

,TEAM_NAME,GP_x,W_x,L_x,W_PCT_x,OFF_RATING_x,DEF_RATING_x,NET_RATING_x,EFG_PCT_x,TS_PCT_x,...,PACE_y,POSS_y,W_RANK_y,L_RANK_y,W_PCT_RANK_y,OFF_RATING_RANK_y,DEF_RATING_RANK_y,NET_RATING_RANK_y,season,Round_x
0,Los Angeles Lakers,19,15,4,0.789,105.2,101.3,3.9,0.482,0.527,...,93.32,7691,2,2,2,2,7,2,2002,4
1,New Jersey Nets,20,11,9,0.550,100.6,100.7,-0.2,0.457,0.503,...,93.20,7707,5,5,5,17,1,5,2002,4
2,Sacramento Kings,16,10,6,0.625,105.0,102.3,2.7,0.478,0.529,...,96.74,7951,1,1,1,3,5,1,2002,3
3,Boston Celtics,16,9,7,0.563,99.9,96.7,3.2,0.469,0.515,...,93.60,7738,8,8,8,18,6,10,2002,3
4,Charlotte Hornets,9,4,5,0.444,101.9,101.5,0.4,0.449,0.505,...,90.93,7480,11,11,11,14,10,13,2002,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,Philadelphia 76ers,6,2,4,0.333,116.9,116.7,0.2,0.525,0.574,...,98.17,8090,12,12,12,14,11,9,2024,1
12,Los Angeles Lakers,5,1,4,0.200,109.7,113.4,-3.7,0.544,0.573,...,101.38,8387,12,12,12,15,17,19,2024,1
13,Miami Heat,5,1,4,0.200,100.7,117.7,-17.0,0.507,0.533,...,96.88,7975,16,16,16,21,5,17,2024,1
14,New Orleans Pelicans,4,0,4,0.000,93.5,109.6,-16.2,0.448,0.489,...,98.70,8101,8,8,8,11,6,6,2024,1


In [30]:
qus = ["Round_x > 0","Round_x > 1","Round_x > 2","Round_x > 3","Round_x > 1 & W_RANK_x == 1"]
scenario = ["Playoffs","2nd Round","Conf Finals","Finals","Championship"]

In [31]:
dfb = []
for q,s in zip(qus,scenario):
    df9 = df8.query(q)
    df9["Scenario"] = s
    dfb.append(df9)
df10 = pd.concat(dfb)

In [32]:
df11 = df10[["TEAM_NAME","season","Scenario"]]